In [1]:
import numpy as np, pandas as pd, polars as pl
from sklearn.model_selection import KFold
from sklearn import preprocessing as skp
from sklearn.decomposition import PCA 
import seaborn as sns

import xgboost as xgb
from sklearn.metrics import root_mean_squared_error as rmse
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('train.csv', index_col = 'id')
test = pd.read_csv('test.csv', index_col = 'id')

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 750000 entries, 0 to 749999
Data columns (total 11 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Podcast_Name                 750000 non-null  object 
 1   Episode_Title                750000 non-null  object 
 2   Episode_Length_minutes       662907 non-null  float64
 3   Genre                        750000 non-null  object 
 4   Host_Popularity_percentage   750000 non-null  float64
 5   Publication_Day              750000 non-null  object 
 6   Publication_Time             750000 non-null  object 
 7   Guest_Popularity_percentage  603970 non-null  float64
 8   Number_of_Ads                749999 non-null  float64
 9   Episode_Sentiment            750000 non-null  object 
 10  Listening_Time_minutes       750000 non-null  float64
dtypes: float64(5), object(6)
memory usage: 68.7+ MB


In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 250000 entries, 750000 to 999999
Data columns (total 10 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Podcast_Name                 250000 non-null  object 
 1   Episode_Title                250000 non-null  object 
 2   Episode_Length_minutes       221264 non-null  float64
 3   Genre                        250000 non-null  object 
 4   Host_Popularity_percentage   250000 non-null  float64
 5   Publication_Day              250000 non-null  object 
 6   Publication_Time             250000 non-null  object 
 7   Guest_Popularity_percentage  201168 non-null  float64
 8   Number_of_Ads                250000 non-null  float64
 9   Episode_Sentiment            250000 non-null  object 
dtypes: float64(4), object(6)
memory usage: 21.0+ MB


In [5]:
categorical_map = {
        'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6, # Day

        'Morning': 0, 'Afternoon': 1, 'Evening': 2, 'Night': 3,                                            # Time

        'Negative': 0, 'Neutral': 1, 'Positive': 2,                                                        # Sentiment

        'True Crime': 0, 'Comedy': 1, 'Education': 2, 'Technology': 3, 'Health': 4, 'News': 5, 'Music': 6, # Genre
        'Sports': 7, 'Business': 8, 'Lifestyle': 9
        }

categories = ['Genre', 'Podcast_Name', 'Episode_Title', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']

In [6]:
podcast = {}
for i, n in enumerate(train['Podcast_Name'].unique()):
    podcast[n] = i

categorical_map.update(podcast)

In [7]:
train['Episode_Title'] = train['Episode_Title'].str[8:]
test['Episode_Title'] = test['Episode_Title'].str[8:]

In [8]:
train = train.replace(categorical_map).fillna(0)
test = test.replace(categorical_map).fillna(0)

In [9]:
x = train.drop('Listening_Time_minutes', axis=1)
y = train['Listening_Time_minutes']

In [10]:
poly = skp.PolynomialFeatures(interaction_only=True, include_bias=False)
x_poly = poly.fit_transform(x)
test_poly = poly.fit_transform(test)

In [11]:
cols = poly.get_feature_names_out(x.columns)
x = pd.DataFrame(x_poly, columns=cols)

test_cols = poly.get_feature_names_out(test.columns)
test = pd.DataFrame(test_poly, columns=test_cols)

In [12]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 55 columns):
 #   Column                                                  Non-Null Count   Dtype  
---  ------                                                  --------------   -----  
 0   Podcast_Name                                            750000 non-null  float64
 1   Episode_Title                                           750000 non-null  float64
 2   Episode_Length_minutes                                  750000 non-null  float64
 3   Genre                                                   750000 non-null  float64
 4   Host_Popularity_percentage                              750000 non-null  float64
 5   Publication_Day                                         750000 non-null  float64
 6   Publication_Time                                        750000 non-null  float64
 7   Guest_Popularity_percentage                             750000 non-null  float64
 8   Number_of_Ads           

In [13]:
important_features_bygain = ['Episode_Length_minutes','Episode_Title Episode_Length_minutes', 'Episode_Title', 'Podcast_Name','Number_of_Ads',
'Episode_Length_minutes Host_Popularity_percentage', 'Podcast_Name Episode_Length_minutes', 'Host_Popularity_percentage']
idx = []
for i, col in enumerate(x.columns): 
    if col in important_features_bygain: idx.append(i)

In [14]:
x_n = x.iloc[idx]
test_n = test.iloc[idx]

In [15]:
for c in categories: 
    x_n[c] = x_n[c].astype('category')
    test_n[c] = test_n[c].astype('category')

C:\Users\evan\AppData\Local\Temp\ipykernel_13008\4176405769.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_n[c] = x_n[c].astype('category')
C:\Users\evan\AppData\Local\Temp\ipykernel_13008\4176405769.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_n[c] = test_n[c].astype('category')


In [16]:
K = 5
kf = KFold(K, shuffle=True, random_state=55)

In [17]:
pred = pd.DataFrame()
pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame


In [ ]:
for train_idx, val_idx in kf.split(x,y):
    x_t, y_t = x.iloc[train_idx], y.iloc[train_idx]
    x_v, y_v = x.iloc[val_idx], y.iloc[val_idx]
    model = xgb.XGBRegressor(objective='reg:squarederror', eval_metric = 'rmse', enable_categorical = True, device = 'cuda', tree_method="hist", min_child_weight=10, learning_rate=0.2, n_estimators=1000, importance_type='gain')
    model.fit(x_t, y_t, eval_set=[(x_v,y_v)])
    pred = pd.concat([pd.DataFrame(model.predict(test)), pred], axis=1)

[0]	validation_0-rmse:21.24222
[1]	validation_0-rmse:17.63194
[2]	validation_0-rmse:15.54909
[3]	validation_0-rmse:14.41017
[4]	validation_0-rmse:13.81063
[5]	validation_0-rmse:13.50164
[6]	validation_0-rmse:13.34275
[7]	validation_0-rmse:13.25999
[8]	validation_0-rmse:13.21413
[9]	validation_0-rmse:13.18931
[10]	validation_0-rmse:13.17317
[11]	validation_0-rmse:13.16325
[12]	validation_0-rmse:13.15392
[13]	validation_0-rmse:13.14745
[14]	validation_0-rmse:13.14260
[15]	validation_0-rmse:13.13887
[16]	validation_0-rmse:13.13596
[17]	validation_0-rmse:13.13306
[18]	validation_0-rmse:13.12661
[19]	validation_0-rmse:13.12403
[20]	validation_0-rmse:13.12095
[21]	validation_0-rmse:13.11595
[22]	validation_0-rmse:13.11345
[23]	validation_0-rmse:13.11147
[24]	validation_0-rmse:13.10930
[25]	validation_0-rmse:13.10869
[26]	validation_0-rmse:13.10677
[27]	validation_0-rmse:13.10320
[28]	validation_0-rmse:13.10266
[29]	validation_0-rmse:13.10121
[30]	validation_0-rmse:13.09986
[31]	validation_0-

c:\Users\evan\anaconda3\lib\site-packages\xgboost\core.py:729: UserWarning: [21:18:42] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)


[0]	validation_0-rmse:21.16720
[1]	validation_0-rmse:17.57018
[2]	validation_0-rmse:15.49624
[3]	validation_0-rmse:14.36435
[4]	validation_0-rmse:13.77043
[5]	validation_0-rmse:13.46693
[6]	validation_0-rmse:13.31098
[7]	validation_0-rmse:13.23016
[8]	validation_0-rmse:13.18722
[9]	validation_0-rmse:13.16135
[10]	validation_0-rmse:13.14650
[11]	validation_0-rmse:13.13822
[12]	validation_0-rmse:13.13052
[13]	validation_0-rmse:13.12453
[14]	validation_0-rmse:13.12041
[15]	validation_0-rmse:13.11493
[16]	validation_0-rmse:13.11103
[17]	validation_0-rmse:13.10867
[18]	validation_0-rmse:13.10681
[19]	validation_0-rmse:13.10509
[20]	validation_0-rmse:13.10295
[21]	validation_0-rmse:13.09620
[22]	validation_0-rmse:13.09410
[23]	validation_0-rmse:13.09266
[24]	validation_0-rmse:13.09220
[25]	validation_0-rmse:13.09067
[26]	validation_0-rmse:13.09009
[27]	validation_0-rmse:13.08752
[28]	validation_0-rmse:13.08524
[29]	validation_0-rmse:13.08243
[30]	validation_0-rmse:13.08127
[31]	validation_0-

In [19]:
pred.reset_index(drop=True, inplace=True)
pred.index += 750000
pred_sum = pred.sum(axis=1) / K

In [20]:
pred.head()

,0,0,0,0,0
750000,54.945656,55.341698,54.949123,55.151604,54.497478
750001,18.330025,17.713345,17.635473,17.534906,18.237627
750002,50.139080,49.402416,49.804783,50.053570,50.988556
750003,77.143127,78.139755,76.419975,79.338371,80.774796
750004,48.734985,48.854565,48.399399,49.191994,49.110626


In [21]:
pred_sum.rename('Listening_Time_minutes', inplace=True)
pred_sum.head()

750000    54.977112
750001    17.890274
750002    50.077682
750003    78.363205
750004    48.858315
Name: Listening_Time_minutes, dtype: float32

In [ ]:
pred_sum.to_csv('submission_newparams2.csv', index_label='id')